# Evaluate Embedding models

### Import necessary modules

In [3]:
import os
import nltk
from transformers import BertTokenizer, BertForMaskedLM, BertModel
from bert_score import BERTScorer
from rouge import Rouge
from torchmetrics.text.bert import BERTScore
from rouge_score import rouge_scorer
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community import embeddings
from nltk.translate.bleu_score import sentence_bleu
from llama_index.core import (
    ServiceContext,
    SimpleDirectoryReader,
    StorageContext,
    VectorStoreIndex,
    set_global_service_context,
)
from langchain.text_splitter import CharacterTextSplitter
from tqdm import tqdm
from llama_index.core import Settings
from llama_index.core.node_parser import SentenceSplitter

c:\Project_Files\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Provide the directory where all the pdfs data are available

In [4]:
pdf_dir = ".\documents\short_story"

In [5]:
pdf_list=os.listdir(pdf_dir)
pdf_list

['Vanka.pdf']

In [6]:
for i, each_pdf in enumerate(pdf_list):
    each_pdf = pdf_dir + '\\' + each_pdf
    pdf_list[i] = each_pdf
pdf_list

['.\\documents\\short_story\\Vanka.pdf']

### Load the data

In [7]:
documents = SimpleDirectoryReader(input_files=pdf_list).load_data()

In [8]:
documents

[Document(id_='18623a01-33fc-4047-a924-12e7f4f8a98d', embedding=None, metadata={'page_label': '1', 'file_name': 'Vanka.pdf', 'file_path': 'documents\\short_story\\Vanka.pdf', 'file_type': 'application/pdf', 'file_size': 195740, 'creation_date': '2024-04-10', 'last_modified_date': '2024-04-10'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text=' \n \n \n \n \n \n \n \n \n \nVanka  \nBy Anton Chekhov   \n ', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='b98283ae-3113-47f1-a023-63e30245380f', embedding=None, metadata={'page_label': '2', 'file_name': 'Vanka.pdf', 'file_path': 'documents\\short_story\\Vanka.pdf', 'file_type': 'application/pdf', 'f

### Extract the content from each pdf document

In [9]:
text_contents = []
for document in documents:
    text_contents.append(document.get_content())
text_contents

[' \n \n \n \n \n \n \n \n \n \nVanka  \nBy Anton Chekhov   \n ',
 'Learn English Through Story  \n \n \n \n \nVanka  \n \n \n \nBy Anton Chekhov  \n \nLevel 5 -6 \n \n \nHope you have enjoyed the reading!  \nCome back to https://learnenglish -new.com/ to find more \nfascinating and exciting stories!  \n \n \nhttps://learnenglish -new.com/  \n ',
 'VANKA ZHUKOV, a boy of nine, who had been for three months apprenticed to \nAlyahin  the shoemaker, was sitting up on Christmas Eve. Waiting till his master and \nmistress and their workmen had gone to the midnight service, he took out of his \nmaster\'s cupboard a bottle of ink and a pen with a rusty nib, and, spreading out a \ncrumpled sheet o f paper in front of him, began writing. Before forming the first letter \nhe several times looked round fearfully at the door and the windows, stole a glance at \nthe dark ikon, on both sides of which stretched shelves full of lasts, and heaved a \nbroken sigh. The paper lay on the bench while he kne

In [10]:
len(text_contents)

7

In [11]:
reference_tokens = [nltk.word_tokenize(reference.lower()) for reference in text_contents]
reference_tokens

[['vanka', 'by', 'anton', 'chekhov'],
 ['learn',
  'english',
  'through',
  'story',
  'vanka',
  'by',
  'anton',
  'chekhov',
  'level',
  '5',
  '-6',
  'hope',
  'you',
  'have',
  'enjoyed',
  'the',
  'reading',
  '!',
  'come',
  'back',
  'to',
  'https',
  ':',
  '//learnenglish',
  '-new.com/',
  'to',
  'find',
  'more',
  'fascinating',
  'and',
  'exciting',
  'stories',
  '!',
  'https',
  ':',
  '//learnenglish',
  '-new.com/'],
 ['vanka',
  'zhukov',
  ',',
  'a',
  'boy',
  'of',
  'nine',
  ',',
  'who',
  'had',
  'been',
  'for',
  'three',
  'months',
  'apprenticed',
  'to',
  'alyahin',
  'the',
  'shoemaker',
  ',',
  'was',
  'sitting',
  'up',
  'on',
  'christmas',
  'eve',
  '.',
  'waiting',
  'till',
  'his',
  'master',
  'and',
  'mistress',
  'and',
  'their',
  'workmen',
  'had',
  'gone',
  'to',
  'the',
  'midnight',
  'service',
  ',',
  'he',
  'took',
  'out',
  'of',
  'his',
  'master',
  "'s",
  'cupboard',
  'a',
  'bottle',
  'of',
  'ink'

In [12]:
len(reference_tokens)

7

### Length of the reference token in page 3

In [13]:
len(reference_tokens[0])

4

### Define the LLM Model

In [14]:
llm = Ollama(model="mistral")

### Define the Embedding model you like to evaluate

In [15]:
model_name = "llama2"

### Store the embedding into the vector database

In [16]:
# Initialize Ollama embeddings.
embeddings = embeddings.ollama.OllamaEmbeddings(model=model_name)
# service_context = (ServiceContext.from_defaults(llm=llm,embed_model=embeddings,chunk_size=300))

In [17]:
Settings.llm = llm
Settings.embed_model = embeddings
Settings.node_parser = SentenceSplitter(chunk_size=256, chunk_overlap=20)
Settings.num_output = 512
Settings.context_window = 3900

# a vector store index only needs an embed model
index = VectorStoreIndex.from_documents(documents, embed_model=embeddings)

In [18]:
index

In [19]:
print("Defining query engine")
query_engine = index.as_query_engine()

Defining query engine


In [20]:
response = query_engine.query("What is the name of Vanka's grandfather?")
response

c:\Project_Files\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Response(response=' Konstantin Makaritch', source_nodes=[NodeWithScore(node=TextNode(id_='c7c4d8d8-53a8-49ae-b70b-7fbf13c07f75', embedding=None, metadata={'page_label': '1', 'file_name': 'Vanka.pdf', 'file_path': 'documents\\short_story\\Vanka.pdf', 'file_type': 'application/pdf', 'file_size': 195740, 'creation_date': '2024-04-10', 'last_modified_date': '2024-04-10'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='18623a01-33fc-4047-a924-12e7f4f8a98d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'Vanka.pdf', 'file_path': 'documents\\short_story\\Vanka.pdf', 'file_type': 'application/pdf', 'file_size': 195740, 'creation_date': '2024-04-10', 'last_modified_date': '2024-04-

### Response:

Response(
    
    response=' Konstantin Makaritch', 
    
    source_nodes=[
        NodeWithScore(
            node=TextNode(
                id_='bf031382-f0dc-4fb9-92ab-d0d889f05982', embedding=None, 
                metadata=
                {'page_label': '3', 
                'file_name': 'Vanka.pdf', 
                'file_path': 'documents\\short_story\\Vanka.pdf', 
                'file_type': 'application/pdf', 
                'file_size': 195740, 
                'creation_date': '2024-04-10', 'last_modified_date': '2024-04-10'}, excluded_embed_metadata_keys=
                ['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=
                ['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], 
                relationships=
                {
                    <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(
                        node_id='181a3548-8499-49fd-8d91-970e4d561a3b', node_type=<ObjectType.DOCUMENT: '4'>, 
                        metadata=
                        {
                            'page_label': '3', 
                            'file_name': 'Vanka.pdf', 
                            'file_path': 'documents\\short_story\\Vanka.pdf', 
                            'file_type': 'application/pdf', 
                            'file_size': 195740, 
                            'creation_date': '2024-04-10', 
                            'last_modified_date': '2024-04-10'
                        }, hash='f8fff45b6dfae5591e44ef56ae2ed2a27d1a316c82a1b1ef8a580350819972a1'
                        ), 
                    <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(
                        node_id='c88126ba-9ab8-4f3a-9a6e-10a9245f4e38', node_type=<ObjectType.TEXT: '1'>, 
                        metadata=
                        {
                            'page_label': '2', 
                            'file_name': 'Vanka.pdf', 
                            'file_path': 'documents\\short_story\\Vanka.pdf', 
                            'file_type': 'application/pdf', 
                            'file_size': 195740, 
                            'creation_date': '2024-04-10', 
                            'last_modified_date': '2024-04-10'
                        }, hash='3957f5b2edb99c0925209a4a75eac5fc4dbc13dfa84956557202cabf4a119384'), 
                    <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(
                        node_id='cd915138-85e8-4cd7-b653-7e277fcf44c2', node_type=<ObjectType.TEXT: '1'>, 
                        metadata=
                        {}, hash='d9bffb01e3671f615d8d468036d388f1586c37e9fd8c7e2480ffcd9bca1a7022')
                }, 
                text='VANKA ZHUKOV, a boy of nine, who had been for three months apprenticed to \nAlyahin  the shoemaker, was sitting up on Christmas Eve. Waiting till his master and \nmistress and their workmen had gone to the midnight service, he took out of his \nmaster\'s cupboard a bottle of ink and a pen with a rusty nib, and, spreading out a \ncrumpled sheet o f paper in front of him, began writing. Before forming the first letter \nhe several times looked round fearfully at the door and the windows, stole a glance at \nthe dark ikon, on both sides of which stretched shelves full of lasts, and heaved a \nbroken sigh. The paper lay on the bench while he knelt before it.  \n \n \n \n"Dear grandfather, Konstantin Makaritch," he wrote, "I am writing you a letter. I wish \nyou a happy Christmas, and all blessings from God Almighty. I have neither father \nnor mother, you are the only o ne left me."', 
                start_char_idx=0, 
                end_char_idx=877, 
                text_template='{metadata_str}\n\n{content}', 
                metadata_template='{key}: {value}', 
                metadata_seperator='\n'
            ), 
            score=0.5509342553884494
        ), 
        NodeWithScore(
            node=TextNode(
                id_='df57241e-359e-4cee-ab2b-231f3cd60d35', 
                embedding=None, 
                metadata={
                    'page_label': '5', 
                    'file_name': 'Vanka.pdf', 
                    'file_path': 'documents\\short_story\\Vanka.pdf', 'file_type': 'application/pdf', 
                    'file_size': 195740, 
                    'creation_date': '2024-04-10', 
                    'last_modified_date': '2024-04-10'
                    }, 
                excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], 
                excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], 
                relationships={
                    <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(
                        node_id='5af0d844-81c1-4a27-8bf0-b526b8a7bed7', node_type=<ObjectType.DOCUMENT: '4'>, 
                        metadata={
                            'page_label': '5', 
                            'file_name': 'Vanka.pdf', 
                            'file_path': 'documents\\short_story\\Vanka.pdf', 'file_type': 'application/pdf', 
                            'file_size': 195740, 
                            'creation_date': '2024-04-10', 'last_modified_date': '2024-04-10'
                        }, hash='1c67b42d300a5b6bcc852112e73c8087d97edc08fb05773c18e7d856fb7c13ec'
                        ), 
                    <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(
                        node_id='03e5420e-eb5a-4d0f-bc18-d3c79b734de6', node_type=<ObjectType.TEXT: '1'>, 
                        metadata={
                            'page_label': '5', 
                            'file_name': 'Vanka.pdf', 
                            'file_path': 'documents\\short_story\\Vanka.pdf', 'file_type': 'application/pdf', 
                            'file_size': 195740, 'creation_date': '2024-04-10', 
                            'last_modified_date': '2024-04-10'}, hash='47303100128855eaae4e30a75e7913440c0d355d2bcb0b7deb0da27ebadbb4a3'
                            ),
                    <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(
                        node_id='b92d4ee1-4ec7-40ca-8c1e-4f4d7a3f05f9', node_type=<ObjectType.TEXT: '1'>, 
                        metadata={}, hash='379886a057226f2fcf3562b071bf7acd8570fc592f750369ba058305af5c564b'
                        )
                    }, 
                text='It\'s \nmore than I can bear. I bow  down to your feet, and will pray to God for you for ever, \ntake me away from here or I shall die."  \nVanka\'s mouth worked, he rubbed his eyes with his black fist, and gave a sob.  \n \n"I will powder your snuff for you," he went on. "I will pray for you, and if I do \nanything you can thrash me like Sidor\'s goat. And if you think I\'ve no job, then I will \nbeg the steward for Christ\'s sake to let me clean his boots, or I\'ll go for a shepherd -\nboy instead of Fedka. Dear grandfather, it is more than I can bear, it\'s simply no life at \nall. I wanted to run away to the village, but I have no boots, and I am afraid of the \nfrost. When I grow up big I will take care of you for this, and not let anyone  annoy \nyou, and when you die I will pray for the rest of your soul, just as for my mammy\'s.', start_char_idx=770, 
                end_char_idx=1599, 
                text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}',
                metadata_seperator='\n'
                ), 
            score=0.4891876337464069
            )
        ], 
        metadata={
            'bf031382-f0dc-4fb9-92ab-d0d889f05982': {
                'page_label': '3', 
                'file_name': 'Vanka.pdf', 
                'file_path': 'documents\\short_story\\Vanka.pdf', 
                'file_type': 'application/pdf', 
                'file_size': 195740, 
                'creation_date': '2024-04-10', 
                'last_modified_date': '2024-04-10'
                }, 
            'df57241e-359e-4cee-ab2b-231f3cd60d35': {
                'page_label': '5', 
                'file_name': 'Vanka.pdf', 
                'file_path': 'documents\\short_story\\Vanka.pdf', 
                'file_type': 'application/pdf', 
                'file_size': 195740, 
                'creation_date': '2024-04-10', 
                'last_modified_date': '2024-04-10'}})

### Get the tokens from the response

In [21]:
candidate_tokens = nltk.word_tokenize(str(response).lower())
candidate_tokens

['konstantin', 'makaritch']

In [22]:
reference_tokens

[['vanka', 'by', 'anton', 'chekhov'],
 ['learn',
  'english',
  'through',
  'story',
  'vanka',
  'by',
  'anton',
  'chekhov',
  'level',
  '5',
  '-6',
  'hope',
  'you',
  'have',
  'enjoyed',
  'the',
  'reading',
  '!',
  'come',
  'back',
  'to',
  'https',
  ':',
  '//learnenglish',
  '-new.com/',
  'to',
  'find',
  'more',
  'fascinating',
  'and',
  'exciting',
  'stories',
  '!',
  'https',
  ':',
  '//learnenglish',
  '-new.com/'],
 ['vanka',
  'zhukov',
  ',',
  'a',
  'boy',
  'of',
  'nine',
  ',',
  'who',
  'had',
  'been',
  'for',
  'three',
  'months',
  'apprenticed',
  'to',
  'alyahin',
  'the',
  'shoemaker',
  ',',
  'was',
  'sitting',
  'up',
  'on',
  'christmas',
  'eve',
  '.',
  'waiting',
  'till',
  'his',
  'master',
  'and',
  'mistress',
  'and',
  'their',
  'workmen',
  'had',
  'gone',
  'to',
  'the',
  'midnight',
  'service',
  ',',
  'he',
  'took',
  'out',
  'of',
  'his',
  'master',
  "'s",
  'cupboard',
  'a',
  'bottle',
  'of',
  'ink'

An embedding is a numerical representation of content in a form that machines can process and understand.
 

The essence of the process is to convert an object, such as an image or text, into a vector that encapsulates its semantic content while discarding irrelevant details as much as possible.
 

Need of Embeddings:

- Primary advantage is that they allow the model to understand the semantic similarity between different pieces of text.
- Encode the input query and the retrieved documents.
- Crucial for effective information retrieval and response generation.


### BLEU Score
- Stands for Bilingual Evaluation Understudy.
- Calculated by comparing the n-grams of your predicted text with the n-grams of the reference text.
- Used for evaluating machine-translation tasks like text-summarization, chatbots, caption generator etc

For more details: https://medium.com/@priyankads/evaluation-metrics-in-natural-language-processing-bleu-dc3cfa8faaa5

### Calculate bleu score

In [23]:
bleu_score = sentence_bleu(reference_tokens, candidate_tokens, weights=(0.25,0.25))
bleu_score

0.36787944117144233

### ROUGE Score

- Metric that compares the output of an NLP model to one or more human reference texts.
- Calculates the overlap between the model output and the reference texts in terms of n-grams, which are sequences of n words.
- Computed for different types of n-grams, such as ROUGE-1, ROUGE-2, or ROUGE-L.

For more details:
https://medium.com/@eren9677/text-summarization-387836c9e178

### Define the rouge score

In [24]:
rouge_score = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

### Calculate ROUGE Score

In [25]:
for ref in text_contents:
    temp_scores = rouge_score.score(ref, str(response))
    print(f'For Reference: {ref} - ROUGE source is {temp_scores}')

For Reference:  
 
 
 
 
 
 
 
 
 
Vanka  
By Anton Chekhov   
  - ROUGE source is {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.0, recall=0.0, fmeasure=0.0)}
For Reference: Learn English Through Story  
 
 
 
 
Vanka  
 
 
 
By Anton Chekhov  
 
Level 5 -6 
 
 
Hope you have enjoyed the reading!  
Come back to https://learnenglish -new.com/ to find more 
fascinating and exciting stories!  
 
 
https://learnenglish -new.com/  
  - ROUGE source is {'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.0, recall=0.0, fmeasure=0.0)}
For Reference: VANKA ZHUKOV, a boy of nine, who had been for three months apprenticed to 
Alyahin  the shoemaker, was sitting up on Christmas Eve. Waiting till his master and 
mistress and their workmen had gone to the midnight service, he took out of his 
master's cupboard 

In [26]:
rouge_scores_list = {key: [] for key in ['rouge1', 'rouge2', 'rougeL']}
rouge_scores_list

{'rouge1': [], 'rouge2': [], 'rougeL': []}

In [27]:
for ref in text_contents:
    temp_scores = rouge_score.score(ref, str(response))
    # print(f'For Reference: {ref} - ROUGE source is {temp_scores}')
    for key in temp_scores:
        rouge_scores_list[key].append(temp_scores[key])
rouge_scores_list

{'rouge1': [Score(precision=0.0, recall=0.0, fmeasure=0.0),
  Score(precision=0.0, recall=0.0, fmeasure=0.0),
  Score(precision=1.0, recall=0.008, fmeasure=0.015873015873015872),
  Score(precision=0.0, recall=0.0, fmeasure=0.0),
  Score(precision=0.0, recall=0.0, fmeasure=0.0),
  Score(precision=0.0, recall=0.0, fmeasure=0.0),
  Score(precision=1.0, recall=0.011834319526627219, fmeasure=0.023391812865497075)],
 'rouge2': [Score(precision=0.0, recall=0.0, fmeasure=0.0),
  Score(precision=0.0, recall=0.0, fmeasure=0.0),
  Score(precision=1.0, recall=0.004016064257028112, fmeasure=0.007999999999999998),
  Score(precision=0.0, recall=0.0, fmeasure=0.0),
  Score(precision=0.0, recall=0.0, fmeasure=0.0),
  Score(precision=0.0, recall=0.0, fmeasure=0.0),
  Score(precision=1.0, recall=0.005952380952380952, fmeasure=0.011834319526627219)],
 'rougeL': [Score(precision=0.0, recall=0.0, fmeasure=0.0),
  Score(precision=0.0, recall=0.0, fmeasure=0.0),
  Score(precision=1.0, recall=0.008, fmeasure=0

In [30]:
mean_precision_rouge_score=0
mean_recall_rouge_score=0
mean_fmeasure_rouge_score=0

num_of_pages = len(rouge_scores_list[key])
for key in rouge_scores_list:
    precision_sum = 0
    recall_sum = 0
    f_sum = 0
    for score in rouge_scores_list[key]:
        precision_sum += score.precision
        recall_sum += score.recall
        f_sum += score.fmeasure
    precision_mean = precision_sum / num_of_pages
    recall_mean = recall_sum / num_of_pages
    fmeasure_mean = f_sum / num_of_pages
    mean_precision_rouge_score+=precision_mean
    mean_recall_rouge_score+=recall_mean
    mean_fmeasure_rouge_score+=fmeasure_mean
    print(f'Mean {key} Score:\tPrecision:{precision_mean}\t Recall:{recall_mean}\tFMeasure:{fmeasure_mean}')
mean_precision_rouge_score/=3
mean_recall_rouge_score/=3
mean_fmeasure_rouge_score/=3

print(f'\nMean ROUGE Score:\tPrecision:{precision_mean}\t Recall:{recall_mean}\tFMeasure:{fmeasure_mean}')


Mean rouge1 Score:	Precision:0.2857142857142857	 Recall:0.0028334742180896023	FMeasure:0.005609261248358992
Mean rouge2 Score:	Precision:0.2857142857142857	 Recall:0.001424063601344152	FMeasure:0.0028334742180896023
Mean rougeL Score:	Precision:0.2857142857142857	 Recall:0.0028334742180896023	FMeasure:0.005609261248358992

Mean ROUGE Score:	Precision:0.2857142857142857	 Recall:0.0028334742180896023	FMeasure:0.005609261248358992


### BERT score
 
- Method used to measure the quality of text summarization.
- Utilizes pre-trained BERT contextual embeddings for both the generated and reference texts, and then calculates the cosine similarity between these embeddings.

For more details: https://medium.com/@abonia/bertscore-explained-in-5-minutes-0b98553bfb71

### Define the BERT score

In [32]:
bert_scorer = BERTScorer(lang="en")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
precision_bert_score_sum = 0
recall_bert_score_sum= 0
f1measure_bert_score_sum = 0

page_sum = 0
for ref in text_contents:
    temp_scores = bert_scorer.score([str(response)], [str(ref)])
    precision_bert_score_sum += temp_scores[0]
    recall_bert_score_sum += temp_scores[1]
    f1measure_bert_score_sum += temp_scores[2]
    page_sum += 1
    print(f"Ref: {page_sum} - Precision: {temp_scores[0]}, Recall: {temp_scores[1]}, F1Measure: {temp_scores[2]}")
    
print(f"\nBERT Score - Precision: {precision_bert_score_sum/page_sum}, Recall: {recall_bert_score_sum/page_sum}, F1Measure: {f1measure_bert_score_sum/page_sum}")

Ref: 1 - Precision: tensor([0.8026]), Recall: tensor([0.7875]), F1Measure: tensor([0.7950])
Ref: 2 - Precision: tensor([0.7661]), Recall: tensor([0.7365]), F1Measure: tensor([0.7510])
Ref: 3 - Precision: tensor([0.8455]), Recall: tensor([0.7367]), F1Measure: tensor([0.7874])
Ref: 4 - Precision: tensor([0.7426]), Recall: tensor([0.7226]), F1Measure: tensor([0.7325])
Ref: 5 - Precision: tensor([0.7370]), Recall: tensor([0.7320]), F1Measure: tensor([0.7345])
Ref: 6 - Precision: tensor([0.7405]), Recall: tensor([0.7266]), F1Measure: tensor([0.7335])
Ref: 7 - Precision: tensor([0.8918]), Recall: tensor([0.7397]), F1Measure: tensor([0.8087])

BERT Score - Precision: tensor([0.7894]), Recall: tensor([0.7402]), F1Measure: tensor([0.7632])


In [ ]:
# def get_bleu_score(reference_tokens, candidate_tokens):
#     # Calculate BLEU score
#     bleu_score = sentence_bleu(reference_tokens, candidate_tokens)
#     return bleu_score

# def get_rouge_score(response):
#     r_score=0
#     for ref in text_contents:
#         temp_scores = r_scorer.score(ref, str(response))
#         for key in temp_scores:
#             r_scores[key].append(temp_scores[key])
#     for key in r_scores:
#         num_scores = len(r_scores[key])
#         #precision_sum = 0
#         #recall_sum = 0
#         f_sum = 0
#         for score in r_scores[key]:
#             #precision_sum += score.precision
#             #recall_sum += score.recall
#             f_sum += score.fmeasure
#         #precision_mean = precision_sum / num_scores
#         #recall_mean = recall_sum / num_scores
#         fmeasure_mean = f_sum / num_scores
#         r_score+=fmeasure_mean
#         #print(f'ROUGE Score:\n{key}:\nPrecision:{precision_mean}          Recall:{recall_mean}          FMeasure:{fmeasure_mean}')
#     r_score= r_score/3
#     return r_score
    
# def get_BERT_score(response, candidate_tokens):
#     tot_precision = 0
#     tot_recall = 0
#     tot_f1Score = 0
#     length=0
#     for ref in text_contents:
#         ref_tokens = nltk.word_tokenize(str(ref).lower())
#         if len(ref_tokens) < 512 and len(candidate_tokens) < 512:
#             temp_scores = scorer.score([str(response)], [str(ref)])
#             tot_precision += temp_scores[0]
#             tot_recall += temp_scores[1]
#             tot_f1Score += temp_scores[2]
#             length += 1
#         '''else:
#             print("Skipping score calculation due to exceeding maximum token length")'''
#     precision = tot_precision / length
#     recall = tot_recall / length
#     f1Score = tot_f1Score / length
#     return precision,recall,f1Score
    
# def store_data_into_vectordb(model_name):    
#     print("Defining the service context")
#     service_context = (
#         ServiceContext
#         .from_defaults(
#             llm=llm,
#             embed_model=embeddings.ollama.OllamaEmbeddings(model=model_name),
#             chunk_size=300
#         )
#     )
#     print("Setting the service context globally")
#     set_global_service_context(service_context)
#     print("Defining the storage context")
#     nodes = (
#         service_context
#         .node_parser
#         .get_nodes_from_documents(documents)
#     )
#     storage_context = StorageContext.from_defaults()
#     storage_context.docstore.add_documents(nodes)
#     print("Indexing the data")
#     index = VectorStoreIndex.from_documents(
#         documents,
#         storage_context=storage_context,
#         llm=llm
#     )
#     print("Finished storing data into vector database")
#     return index
    
# def calculate_score(model_name):
#     index = store_data_into_vectordb(model_name)
    
#     print("Defining query engine")
#     query_engine = index.as_query_engine()
#     response = query_engine.query("What are the types of correlation?")
#     print("Embedding Model:", model_name)
    
#     candidate_tokens = nltk.word_tokenize(str(response).lower())
    
#     bleu_score = get_bleu_score(reference_tokens, candidate_tokens)
#     print("BLEU Score:", bleu_score)
    
#     r_score = get_rouge_score(response)
#     print(f'ROUGE Score:{r_score}')
    
#     # BERT Score calculation
#     precision,recall,f1Score = get_BERT_score(response, candidate_tokens)
#     print(f"BERT Score:\nPrecision: {precision}    Recall: {recall}     F1-Score: {f1Score}\n\n\n")
    
# #Mistral
# calculate_score("mistral")
# #llama2
# calculate_score("llama2")
# #gemma
# calculate_score("gemma")
# #llava
# calculate_score("llava")
# #mixtral
# calculate_score("mixtral")